<a href="https://colab.research.google.com/github/lima-breno/natural_language_processing/blob/main/Sumarizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yaml
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup
import requests
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import operator
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Sumarização

##Coleta do dado

In [ ]:
noticia_url = "https://g1.globo.com/tecnologia/noticia/2022/03/12/instagram-restrito-na-russia-entenda-a-importancia-da-rede-social-para-o-pais-de-putin.ghtml"

In [ ]:
def match_class(target):
    def do_match(tag):
        classes = tag.get('class', [])
        return all(c in classes for c in target)
    return do_match

def get_text_url(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    #remove marcações de scripts e style
    texto = soup.find_all(match_class(["content-text__container"]))
    all_text = ""
    for t in texto:
        all_text += t.get_text()
    return all_text

In [ ]:
texto_noticia = get_text_url(noticia_url)

In [ ]:
texto_noticia

## Pré-processamento

In [ ]:
len(set(texto_noticia.split()))

In [ ]:
len(texto_noticia)

In [ ]:
nltk.download('punkt_tab')
corpus_sent = sent_tokenize(texto_noticia)

In [ ]:
corpus_sent

In [ ]:
nltk.download('stopwords')

In [ ]:
corpus_sent

In [ ]:
def pre_processamento_texto(corpus):
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    corpus_alt = [t.lower() for t in corpus_alt]
    portugues_stops = stopwords.words('portuguese')
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]

    return corpus_alt

In [ ]:
corpus_processado = [pre_processamento_texto(sent) for sent in corpus_sent]

In [ ]:
corpus_processado[0]

In [ ]:
vocabulario = set()
for corpus in corpus_processado:
    vocabulario.update(set(corpus))

In [ ]:
len(vocabulario)

Para realizar a sumarização iremos criar os seguintes métodos;

1. Um que calcula a similaridade de frases. Usando Bow
2. Iremos construir uma matriz de similaridade
3. Iremos fazer o rank das frases utilizando o método pagerank. Este método utiliza da representação de grafos então transformaremos nosso dado em um grafo.
4. Ordene o score e retorne os 5 primeiros.

In [ ]:
def similarity_sentences(sent1, sent2):
    all_words = list(set(sent1 + sent2))
    vect_bag = CountVectorizer(binary=False, analyzer="word")
    vect_bag.fit(all_words)
    #aplica BOW
    vec_sent1 = np.asarray(vect_bag.transform([' '.join(sent1)]).todense())
    vec_sent2 = np.asarray(vect_bag.transform([' '.join(sent2)]).todense())

    return cosine_similarity(vec_sent1, vec_sent2).reshape(-1)[0]

In [ ]:
def matrix_similarity(sentences):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for ix in range(0,len(sentences)):
        for ix2 in range(0,len(sentences)):
            if ix == ix2:
                continue
            similarity_matrix[ix][ix2] = similarity_sentences(sentences[ix], sentences[ix2])
    return similarity_matrix

In [ ]:
def get_pagerank(texto):
    sentences_matrix_similarity = matrix_similarity(texto)
    sentence_similarity_graph = nx.from_numpy_array(sentences_matrix_similarity)
    scores = nx.pagerank(sentence_similarity_graph)

    return scores

In [ ]:
def summarize(scores):
    scores_sorted = dict(sorted(scores.items(), key=operator.itemgetter(1),reverse=True))
    rank_sentences = list(scores_sorted.keys())[:10]
    summarize_text = ""
    for r in rank_sentences:
        summarize_text += (corpus_sent[r]) + " "
    return summarize_text

In [ ]:
summarize(get_pagerank(corpus_processado))

In [ ]:
## Trocando para tfidf
def similarity_sentences(sent1, sent2):
    all_words = list(set(sent1 + sent2))
    vect = TfidfVectorizer()

    vect.fit(all_words)
    vec_sent1 = np.asarray(vect.transform([' '.join(sent1)]).todense())
    vec_sent2 = np.asarray(vect.transform([' '.join(sent2)]).todense())

    return cosine_similarity(vec_sent1, vec_sent2).reshape(-1)[0]

In [ ]:
summarize(get_pagerank(corpus_processado))